# Stock Data Analysis

Identify major events / anomalies through the years
Are some stocks correlated with others?
Are returns normally distributed?
Is the price behavior stationary?
Are there columns without any missing data?
Which stocks are most correlated with the market as a whole?
Is there decay in time-autocorrelation functions?
Are stock prices today correlated with yesterday? Two days ago? A year ago? Ten years ago?
What are seasonality effects? Day of week, week of month, month of year?

To work on: is price at a given time normally distributed?
Is log return at a given time normally distributed?

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from sklearn import linear_model
%matplotlib inline
#pd.options.display.float_format = '{:.5f}'.format
from datetime import datetime
import math

# Suppress annoying harmless error.
import warnings
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

## Daily Closing Price
#### Import csv and clean

In [16]:
df = pd.read_csv('stocks-us-adjClose.csv')
df.rename(columns={'compiled from Yahoo! Finance data by Matt Borthwick':'Date'}, inplace=True)
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
df.index = df.loc[:,'Date']
df.drop(['Date'], axis=1, inplace=True)
display(df.head())

,ED,DD,CVX,FL,CAT,IP,SJW,F,LLY,AVP,...,EXTN,VYGR,ACG,MIME,TCRZ,MCX,EDIT,LMHA,UA,BTU
Date,,,,,,,,,,,,,,,,,,,,,
1970-01-02,0.307997,0.000480,0.582503,1.857836,1.475225,1.808415,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-01-05,0.320831,0.000481,0.585290,1.839623,1.470783,1.873423,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-01-06,0.316553,0.000477,0.576929,1.845694,1.435234,1.855694,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-01-07,0.312275,0.000475,0.575535,1.809266,1.390800,1.814324,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-01-08,0.312275,0.000469,0.586683,1.821408,1.395244,1.832054,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Daily Log Return
#### Creates log-return data frame

In [15]:
logReturns = lambda x: np.log(x) - np.log(x.shift(1))
df_returns =  df.iloc[:,:].apply(logReturns)
df_returns.index = df.index
#df_returns.fillna(1000.0, inplace=True)
display(df_returns.head())

,ED,DD,CVX,FL,CAT,IP,SJW,F,LLY,AVP,...,EXTN,VYGR,ACG,MIME,TCRZ,MCX,EDIT,LMHA,UA,BTU
Date,,,,,,,,,,,,,,,,,,,,,
1970-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-01-05,0.040824,0.002081,0.004773,-0.009852,-0.003016,0.035316,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-01-06,-0.013424,-0.008351,-0.014388,0.003295,-0.024467,-0.009508,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-01-07,-0.013606,-0.004202,-0.002419,-0.019934,-0.031449,-0.022546,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-01-08,0.000000,-0.012712,0.019185,0.006689,0.003190,0.009725,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Annual Volatility = root 252 times standard deviation of daily returns

In [4]:
volatility = df_returns.std()*math.sqrt(252)
display(volatility[volatility > 1.0])

DD       6.741374
MUX      1.118316
SUN      9.825225
ARWR     2.084353
MED      1.247039
CETV     1.120529
AGX      1.035308
BST      5.494214
GURE     1.004254
CTAA    20.133105
SOL      1.019968
GBT      1.021346
BCEI     1.071656
PTCT     1.045742
NORD     1.525742
dtype: float64

In [17]:
corr_array = np.array(df_returns.corr()).ravel()
df_array = np.array(df_returns).ravel()

In [21]:
list(corr_array)

[1.0,
 0.019762828902441101,
 0.26471187271111246,
 0.15128259450278569,
 0.21412684562132012,
 0.23490665653607212,
 0.17060693417107653,
 0.20705608471738657,
 0.24265397608259553,
 0.21001196881899767,
 0.24487832343765539,
 0.17145602826463896,
 0.17005738012600588,
 0.23406224581953655,
 0.16923679909008382,
 0.22816674821372854,
 0.23923887353518444,
 0.23879636595688819,
 0.28847560822749957,
 0.18729197091813096,
 0.16877497644419664,
 0.15166718908853244,
 0.12344446068592664,
 0.14905092532156219,
 0.14989169481666176,
 0.24974592380097138,
 0.21826638411804647,
 0.22520369793401102,
 0.23278806121450532,
 0.20099092038207023,
 0.13235093081355931,
 0.20171367597323941,
 0.17202128195514421,
 0.17912580911128176,
 0.25618053791122458,
 0.2927701376822438,
 0.13498695870399741,
 0.19277209586385738,
 0.064872408957882571,
 0.30811574362778404,
 0.14758027616235608,
 0.24075564027452759,
 0.14987476476543862,
 0.18411766598315604,
 0.13898664944142591,
 0.16859354488511144,
 0.

Plot of correlation coefficients for all log returns

In [ ]:
pd.Series(corr_array[corr_array!=1]).hist(bins=500)
#plt.xline
plt.show()
from scipy import stats
print(stats.describe(corr_array))

In [ ]:
pd.Series(df_array).hist(bins=5000)
plt.show()
print(stats.describe(df_array))

In [ ]:
#plt.figure(figsize=(14,6))

#plt.subplot(1,1,1)
#df_returns[['CVX','BP', 'OXY']].plot.hist(stacked=True, bins=200)
#plt.axis([-.1,.1,0,2000])

#plt.subplot(1,3,2)
#df_subset['log_ret_BP'].plot.hist(bins=200, color='green')
#plt.axis([-.1,.1,0,2000])

#plt.subplot(1,3,3)
#df_subset['log_ret_OXY'].plot.hist(bins=200, color='red')
#plt.axis([-.1,.1,0,2000])

#plt.show()

In [ ]:
plt.figure(figsize=(13,10))

ax1 = plt.subplot(3,1,1)
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.get_xaxis().tick_bottom()
ax1.get_yaxis().tick_left()
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('Date', fontsize=16)
plt.ylabel('Log Return', fontsize=16)
plt.plot(df_returns.loc[df_returns['AGX']!=1000, 'AGX'], color='blue')
#plt.ylim([-.5,.5])
plt.title('Argan, Inc. AGX', fontsize=20)

ax2 = plt.subplot(3,1,2)
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax2.get_xaxis().tick_bottom()
ax2.get_yaxis().tick_left()
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('Date', fontsize=16)
plt.ylabel('Log Return', fontsize=16)
plt.plot(df_returns.loc[df_returns['CTAA']!=1000, 'CTAA'], color='green')
#plt.ylim([-.5,.5])
plt.title('Qwest CTAA', fontsize=20)

ax3 = plt.subplot(3,1,3)
ax3.spines['top'].set_visible(False)
ax3.spines['right'].set_visible(False)
ax3.get_xaxis().tick_bottom()
ax3.get_yaxis().tick_left()
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('Date', fontsize=16)
plt.ylabel('Log Return', fontsize=16)
plt.plot(df_returns.loc[df_returns['SUN']!=1000, 'SUN'], color='red')
#plt.ylim([-.5,.5])
plt.title('Sunoco SUN', fontsize=20)

plt.tight_layout()
plt.show()